In [19]:
using Test
using Plots
using Printf
using Unitful

In [2]:
import Unitful:
    nm, μm, mm, cm, m, km,
    mg, g, kg,
    ns, μs, ms, s, minute, hr, d, yr, Hz,
    eV,
    μJ, mJ, J,
	μW, mW, W

In [36]:
include("../src/LaserLab.jl")

Main.LaserLab

# Gaussian Laser

### Gaussian beam formulas

$g(r,z) = e^{-2(r/w(z))^2}$

$w(z) = w_0 \sqrt{1 + (\frac{z}{z_r})^2}$

$I(r,z) = I_0 [\frac{w_0}{w(z)}]^2 g(r,z)$

$z_r = \pi  \frac{w_0^2}{\lambda}$

The formulas verify the following limits:

$I(r, 0) = I_0 e^{-2(r/w_0)^2}$

$I(0, z) = I_0 [\frac{w_0}{w(z)}]^2 = \frac{I_0}{1 + (z/z_r)^2}$

Which has maximum value ($I_0$) at $z=0$ and then decreases as z increases. For $z=z_r$ the intensity along the axis is reduced by half. 

for $z << z_r, w(z) = w_0$, and:

$I(0, z) = I_0$

$I(r, z) = I_0 e^{-2(r/w_0)^2}$


Thus, the beam intensity falls exponentially with the radius r and is a constant ($I_0$) along the longitudinal axis, for $r=0$.

The beam power is obtained integrating the irradiance at any transverse plane. For simplicity we can choose $z=0$:

$P = \int_0^\infty I(r,0) 2 \pi r dr = 2 \pi I_0 \int e^{-2(r/w_0)^2} r dr$ 

Define $y = \frac{2 r^2}{w_0^2}$. Then $dy = \frac{4 r}{w_0^2} dr$, $dr = \frac{w_0^2}{4 r} dy$ and:

$P = 2 \pi I_0 \frac{w_0^2}{4} \int_0^\infty e^{-y} dy$ 

Since $\int_0^\infty e^-y dy = 1$, we find:

$P = I_0 \frac{\pi w_0^2}{2}$

or:

$I_0 = \frac{2 P}{\pi w_0^2}$ 


### Define laser and objective 

In [47]:
luv = LaserLab.Laser(375.0nm, 1.0μW)
lmu_40x_nuv = LaserLab.Objective("LMU-40X-NUV", 5.3mm, 5.1mm, 40.0)


Main.LaserLab.Objective("LMU-40X-NUV", 0.48113207547169806, 40.0, 5.3 mm, 5.1 mm)

	GLaser

Representation of a Gaussian laser

### Fields
- `λ::typeof(1.0nm)`       : Laser wavelength
- `P::typeof(1.0mW)`       : Power
- `w0::typeof(1.0nm)`      : Waist of laser at focusing point
- `zr::typeof(1.0nm)`      : Rayleigh length
- `I0::typeof(1.0mW/cm^2)` : Intensity at r = 0, z= 0
- `ρ0::typeof(1.0Hz/cm^2)` : Photon density at r = 0, z= 0

In [48]:
gl  = LaserLab.GaussianLaser(luv,lmu_40x_nuv)
@printf("λ = %5.1f nm P = %5.1f mW, w0 = %5.1f μm, I(0,0) = %5.2g mW/cm2 ρ(0,0) = %5.2g Hz/cm2", 
        uconvert(nm, gl.laser.λ)/nm, 
        uconvert(μW, gl.laser.P)/μW, 
        uconvert(nm, gl.w0)/nm, gl.I0/(mW*cm^-2), gl.ρ0/(Hz*cm^-2))


λ = 375.0 nm P =   1.0 mW, w0 = 248.1 μm, I(0,0) = 1e+06 mW/cm2 ρ(0,0) = 9.8e+20 Hz/cm2

In [49]:
gl.zr

515.6473300405238 nm

### Calibration setup

Consider now the problem of calibrating the TOPATU setup using a fluorescent solution of known fluorophores cross section (σ) and quantum yield (Q), with a given concentration (C). The solution is contained in a vial of quartz. The laser beam is collimated to a waist of $(gl.w0)$ and crosses the vial, which has a thickness $(lz)$. A CCD/PMT with a lens of diameter $d_l$ located at a position $x_l$, with respect to the vial, along the z axis, records the photons emitted by the solution. 

For a waist of $(gl.w0)$, $gl.zr$ = $(lti.LabTools.to_fstr(gl.zr/mm,"%5.2g")) and thus any point in the vial has a small z compared with zr. Therefore:

$w(z) = w_0$

$I(r, z) = I_0 e^{-2(r/w_0)^2}$

When the beam crosses the vial it iluminates all the fluorophores. The emitted rate of photons is found by integrating the irradiance and multiplying by the cross section, the quantum yield and the concentration:

$N = \sigma\, Q\, C \int_z \int_r I(r) 2 \pi r dr$

The integral in $z$ is trivial and yields the thickness of the vial, $t$. The integral in $r$ can be precisely approximated by the integral to all the space, since the dimensions in r of the vial are much larger than the beam spread. Thus:

$N = I_0\, \sigma\, Q\, C\, t\, 2 \pi \int e^{-2(r/w_0)^2} r dr$

$N =  \frac{ I_0\, \sigma\, Q\, C\, t\, \pi w_0^2}{2}$

For a gaussian beam > 99.9 % of the power is contained in a radius $\rho = 2 w(z)$. In our case $w(z) \sim w_0$, and thus all the beam power is contained in $2 w_0$. Thus:

- Beam power contained in $(2*gl.w0)

This means that the photons are emitted along a "wire" of $(2*gl.w0) mm diameter and  $lz length. The dimensions of the fluorescent volume are small compared with the solid angle, since the CCD is at a distance d = $dccd mm much larger than t = $lz mm. Therefore, we can approximate that the emission occurs from a single point located in the center of the vase. 

The geometrical acceptance is: 

$A = \frac{1}{2}(1 - \frac{d}{\sqrt{(d^2 + (D/2)^2}})$

where $D$ is the aperture of the CCD and $d$ is the distance to the interaction point.

In [ ]:
@test fov.a ≈ π * (fov.d/2.)^2 
@test fov.v ≈ fov.a * fov.z

	struct Laser

Simple representation of a laser

#### Fields
- `λ::typeof(1.0nm)`  : Laser wavelength
- `P::typeof(1.0mW)`  : Power

In [ ]:
luv = LaserLab.Laser(375.0nm, 1.0μW)

	Objective

Simple representation of a microscope objective

#### Fields
- `name::String` : identifies the objective
- `NA::Float64`  : Numerical aperture
- `M::Float64`   : Magnification

In [ ]:
obj = LaserLab.Objective("Generic", 0.5, 40.0)

	GaussianLaser

Representation of a Gaussian laser

#### Fields
- `laser::Laser`       : A laser type
- `obj::Objective`     : An objective type
- `w0::typeof(1.0nm)`  : Waist of laser at focusing point
- `zr::typeof(1.0nm)`  : z of laser at focusing point

In [ ]:
gl = LaserLab.GaussianLaser(luv,obj)

In [ ]:
@test gl.w0 ≈ luv.λ/(π * obj.NA)
@test gl.zr ≈ luv.λ/(π * obj.NA^2)


	photon_energy(λ::Unitful.Length)

Given wavelength of photon return its energy.
#### Fields

- `λ::Unitful.Length`  : Photon wavelength


In [ ]:
LaserLab.photon_energy(500.0nm)

In [ ]:
function hnu_eV(lambda_nm)
    return 1240.0 * (1.0/lambda_nm)
end


In [ ]:
@test isapprox(LaserLab.photon_energy(500.0nm)/eV, hnu_eV(500.0), rtol=0.01)

	delivered_energy(laser::Laser, t::Unitful.Time)

Delivered energy of a laser in a given time.
#### Fields

- `laser::Laser`     : Laser
- `t::Unitful.Time`  : Time in which target is illuminated


In [ ]:
uconvert(J, LaserLab.delivered_energy(luv, 1*s)) ≈ 1.0μJ

	n_photons(laser::Laser)

Rate of photons (number of photons per unit time) produced by a laser
#### Fields

- `laser::Laser`     : Laser
- `t::Unitful.Time`  : Time in which target is illuminated


In [ ]:
LaserLab.n_photons(luv)

    n_photons(λ::Unitful.Length, p::Unitful.Power)

Rate of photons (number of photons per unit time) corresponding to a wavelength
λ and a power P

#### Fields

- `λ::Unitful.Length` : photon wavelength
- `p::Unitful.Power`  : Power

In [ ]:
LaserLab.n_photons(luv) ≈ LaserLab.n_photons(375.0nm, 1.0μW)

	n_photons_int(laser::Laser, t::Unitful.Time)

Integrated number of photons in a given time emitted by a laser

#### Fields

- `laser::Laser`    : Laser
- `t::Unitful.Time` : time of measurement

In [ ]:
LaserLab.n_photons_int(luv, 1.0s) ≈ LaserLab.n_photons(luv)/Hz

    photon_density(λ::Unitful.Length, p::Unitful.Power, a::Unitful.Area)

number of photons per unit time per unit area

#### Fields

- `λ::Unitful.Length` : photon wavelength
- `p::Unitful.Power`  : Power
- `a::Unitful.Area`   : Area

In [ ]:
LaserLab.photon_density(375.0nm, 1.0μW, fov.a)

	photon_density(l::Laser, fov::Fov)

number of photons per unit time per unit area, in a Fov illuminated by a laser

#### Fields

- `laser::Laser` : Laser
- `fov::Fov`     : Field of view


In [ ]:
LaserLab.photon_density(375.0nm, 1.0μW, fov.a) ≈ LaserLab.photon_density(luv, fov)


	w(gl::GaussianLaser, z::Unitful.Length)

Waist of a laser at length z

#### Fields

- `gl::GaussianLaser`  : A gaussian laser
- `z::Unitful.Length`  : z distance from focusing point


In [ ]:
LaserLab.w(gl, gl.zr)

In [ ]:
pol3(x, a, b, c, d) = a + b*x + c*x^2 + d*x^3
x=collect(LinRange(0., 10., 100))
y = pol3.(x,(5.0,), (3.0,), (1.5,),(0.75,),)
plot(x,y, label="y = 5 + 3 x + 1.5 x^2 + 0.75 x^3")
xlabel!("x")
ylabel!("y")  